# 🚛 Mantenimiento Predictivo de Flotas con Redes Neuronales
Este cuaderno aplica un modelo de red neuronal artificial para predecir si un vehículo requiere mantenimiento próximo, basado en variables operativas como kilometraje, horas de motor, temperatura, vibración, entre otros.

## 📚 Paso 1: Importar librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

## 📂 Paso 2: Cargar archivo CSV con datos de flota

In [ ]:
# Asistente para cargar archivo CSV desde el equipo
from google.colab import files
import io

uploaded = files.upload()
for nombre in uploaded.keys():
    df = pd.read_csv(io.BytesIO(uploaded[nombre]))
    break
df.head()

## 🔍 Paso 3: Preprocesamiento de los datos

In [ ]:
# Separar características y etiqueta
X = df.drop(columns=['Vehiculo_ID', 'Falla_proxima'])
y = df['Falla_proxima']

# Escalar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Separar datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## 🧠 Paso 4: Construcción del modelo de red neuronal

In [ ]:
# Definir modelo secuencial simple
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar modelo
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=0)

## 📈 Paso 5: Evaluar desempeño del modelo

In [ ]:
# Curva de precisión durante el entrenamiento
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión del modelo')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Predicciones y métricas
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

# Matriz de confusión
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.show()